In [2]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Broste Thesis Data ##

In [10]:
summary = Table(['Age', 'Condition', 'Total', 'Deaths', 'CHD Deaths']).with_rows([
    ['0-34',  'Diet',    1367, 3, 0],
    ['35-44', 'Diet',    728, 3, 0],
    ['45-54', 'Diet',    767, 14, 4],
    ['55-64', 'Diet',    870, 35, 7],
    ['65+',   'Diet',    953, 190, 42],
    ['0-34',  'Control', 1337, 7, 1],
    ['35-44', 'Control', 731, 4, 1],
    ['45-54', 'Control', 816, 16, 4],
    ['55-64', 'Control', 896, 33, 12],
    ['65+',   'Control', 958, 162, 34],   
])
summary

Age,Condition,Total,Deaths,CHD Deaths
0-34,Diet,1367,3,0
35-44,Diet,728,3,0
45-54,Diet,767,14,4
55-64,Diet,870,35,7
65+,Diet,953,190,42
0-34,Control,1337,7,1
35-44,Control,731,4,1
45-54,Control,816,16,4
55-64,Control,896,33,12
65+,Control,958,162,34


In [12]:
subjects = Table(['Age', 'Condition', 'Participated', 'Died'])
for row in summary.rows:
    i = np.arange(0, row.item('Total'))
    t = Table().with_columns('Died', i < row.item('Deaths'))
    t.append_column('Age', row.item('Age'))
    t.append_column('Condition', row.item('Condition'))
    t.append_column('Participated', True)
    subjects.append(t)
subjects

Age,Condition,Participated,Died
0-34,Diet,True,True
0-34,Diet,True,True
0-34,Diet,True,True
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False


In [13]:
subjects.group(['Age', 'Condition'], sum)

Age,Condition,Participated sum,Died sum
0-34,Control,1337,7
0-34,Diet,1367,3
35-44,Control,731,4
35-44,Diet,728,3
45-54,Control,816,16
45-54,Diet,767,14
55-64,Control,896,33
55-64,Diet,870,35
65+,Control,958,162
65+,Diet,953,190


In [14]:
subjects.group('Condition', sum)

Condition,Age sum,Participated sum,Died sum
Control,,4738,222
Diet,,4685,245


In [15]:
subjects.drop('Age').group('Condition', sum)

Condition,Participated sum,Died sum
Control,4738,222
Diet,4685,245


In [16]:
def hazard_rate(counts):
    return counts.item('Died sum') / counts.item('Participated sum')

def rate_difference(t):
    counts = t.drop('Age').group('Condition', sum)
    return abs(hazard_rate(counts.row(1)) - hazard_rate(counts.row(0)))

rate_difference(subjects)

0.005439343927004493

In [17]:
rate_difference(subjects.where('Age', '0-34'))

0.0030410154080667343

In [18]:
rate_difference(subjects.where('Age', '65+'))

0.030268112783058437

In [19]:
def test(t):
    observed = rate_difference(t)
    repetitions = 200

    stats = make_array()
    for i in np.arange(repetitions):
        simulated_results = t.select('Died').sample(with_replacement=False).column('Died')
        simulated_outcomes = t.with_column('Died', simulated_results)
        simulated_stat = rate_difference(simulated_outcomes)
        stats = np.append(stats, simulated_stat)

    # Find the empirical P-value:
    p = np.count_nonzero(stats >= observed) / repetitions
    
    print('Observed absolute difference in hazard rates:', observed)
    print('P-value:', p)

test(subjects)

Observed absolute difference in hazard rates: 0.005439343927004493
P-value: 0.215


In [20]:
subjects.group('Age').column('Age')

array(['0-34', '35-44', '45-54', '55-64', '65+'], dtype='<U5')

In [21]:
for age in subjects.group('Age').column('Age'):
    print('Ages', age)
    test(subjects.where('Age', age))

Ages 0-34
Observed absolute difference in hazard rates: 0.0030410154080667343
P-value: 0.205
Ages 35-44
Observed absolute difference in hazard rates: 0.0013510771034710841
P-value: 1.0
Ages 45-54
Observed absolute difference in hazard rates: 0.0013549096300841078
P-value: 0.915
Ages 55-64
Observed absolute difference in hazard rates: 0.00339952791461412
P-value: 0.67
Ages 65+
Observed absolute difference in hazard rates: 0.030268112783058437
P-value: 0.1
